# Delete this after ward

In [2]:
import pandas as pd
import seaborn as sns
import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

import tensorflow as tf
# import tensorflow.compat.v1 as tf
from tensorflow import keras
from tensorflow.keras.layers import Lambda
from keras.preprocessing.sequence import pad_sequences
import os, sys, time
from math import floor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from collections import OrderedDict
# from tensorflow.python.ops import math_ops
# import tensorflow.keras.backend as kb
from tensorflow_addons.utils.ensure_tf_install import _check_tf_version
_check_tf_version()
from tensorflow_addons import losses
# Custom Functions
# Added to path by using sys.path.append('')
# sys.path.append('/home/akanu/git/anomalous_pred/custom_functions')
# sys.path.insert(0,'/home/akanu/git/anomalous_pred/custom_functions')
# sys.path.append('/mnt/roahm/users/akanu/git/anomalous_pred/custom_functions')
from load_data import Files_Load, Boxes, test_split_norm_abnorm, norm_train_max_min
from custom_metrics import bb_intersection_over_union, bb_intersection_over_union_np
from coordinate_change import xywh_tlbr, tlbr_xywh
from pedsort import pedsort
from load_data_binary_class import return_indices, binary_data_split, same_ratio_split_train_val
from load_data_binary_class import one_weight_ratio_train
from TP_TN_FP_FN import seperate_misclassifed_examples, sort_TP_TN_FP_FN_by_vid_n_frame
# from plot_all_videos import cycle_through_videos

# Kmeans
from sklearn.cluster import KMeans

# Need to figue out why this is not working
# from persistence1d import RunPersistence


from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, precision_recall_curve

In [23]:
frames = 20

train_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Train_Box/"
test_file = "/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Test_Box/"

loc_files_train, loc_files_test, box_train_txt, box_test_txt = Files_Load(train_file, test_file)

# Don't forget to change to xywh 
traindict = Boxes(loc_files_train, box_train_txt, frames, pad ='pre', to_xywh=True)
testdict = Boxes(loc_files_test, box_test_txt, frames, pad ='pre', to_xywh = True)
abnormal_dict, normal_dict = test_split_norm_abnorm(testdict) #splits by which predicted frames are normal or not

# Normilize data
max1 = traindict['x_ppl_box'].max()
min1 = traindict['x_ppl_box'].min()
xx,yy = norm_train_max_min(data_dict = traindict, max1=max1,min1=min1)
xx_norm,yy_norm = norm_train_max_min(data_dict = normal_dict, max1=max1,min1=min1)
xx_abnorm,yy_abnorm = norm_train_max_min(data_dict = abnormal_dict, max1=max1,min1=min1)


In [24]:
pd.set_option('display.max_rows', 500)

In [32]:
Data = pd.read_csv(loc_files_test[2:3][0], ' ')

In [33]:
loc_files_test[2:3][0]

'/home/akanu/Dataset/Anomaly/Avenue_Dataset/bounding_box_tlbr/Txt_Data/Test_Box/03.txt'

In [34]:
Data.head(50)

,Frame_Number,Person_ID,State,BB_tl_0,BB_tl_1,BB_br_0,BB_br_1,anomaly
0,0,1,1,207,101,252,247,0
1,0,2,1,316,125,346,219,0
2,0,3,1,170,115,212,251,0
3,0,4,1,48,111,87,237,0
4,0,5,1,81,116,120,232,0
5,0,6,1,384,121,423,224,0
6,0,7,1,500,122,514,221,0
7,1,1,1,209,101,254,247,0
8,1,2,1,314,125,344,218,0
9,1,3,1,172,115,214,251,0


In [35]:
Data[Data['Person_ID'] == 32 ]

,Frame_Number,Person_ID,State,BB_tl_0,BB_tl_1,BB_br_0,BB_br_1,anomaly
2699,294,32,1,599,101,637,242,1
2706,295,32,1,586,96,625,239,1
2713,296,32,2,576,94,618,241,1
2720,297,32,2,571,92,613,241,1
2727,298,32,2,565,91,608,242,1
2734,299,32,2,560,89,603,242,0
2741,300,32,2,555,88,598,242,0
2747,301,32,2,486,99,525,239,1
2753,302,32,2,468,96,508,235,1
2759,303,32,2,458,131,492,234,1
